# Water Quality Monitoring - Exploratory Data Analysis
```L. Patterson & John Fay
Spring 2018```

In this notebook, we examine the two datsets generated from our query of the [Water Quality Data portal](https://www.waterqualitydata.us/portal/) for nutrient data in HUC 03030002 from 1970 trough 2017. The first dataset ([sites.csv](https://www.waterqualitydata.us/Station/search?countrycode=US&statecode=US%3A37&siteType=Lake%2C%20Reservoir%2C%20Impoundment&siteType=Stream&huc=03030002&sampleMedia=Water&characteristicType=Nutrient&minresults=500&startDateLo=01-01-1970&startDateHi=12-31-2017&mimeType=csv&zip=yes&sorted=no)) contains data on the locations where samples were collected, and the second dataset ([results.csv](https://www.waterqualitydata.us/Result/search?countrycode=US&statecode=US%3A37&siteType=Lake%2C%20Reservoir%2C%20Impoundment&siteType=Stream&huc=03030002&sampleMedia=Water&characteristicType=Nutrient&minresults=500&startDateLo=01-01-1970&startDateHi=12-31-2017&mimeType=csv&zip=yes&sorted=no)) contains data on the nutrient samples collected at these sites. 

Our goals here are to upload, prepare (tidy, explore, and merge) these data, and ultimately provide some visualizations that reveal the state of water quality in Jordan lake. We'll begin with the **sites** dataset, examining a few attributes about these data, and the move to the **results** where we filter our data for 'clean' records and tidy it for analysis.

## Prep the workspace

In [ ]:
#Import modules
import pandas as pd
import folium
from matplotlib import pyplot as plt

#Enable notebook plotting
%matplotlib inline

## Import and analyzie the water quality *sites* data

### Read the data in as a Pandas dataframe

In [ ]:
#Read in the data into the 'sites' dataframe 
sites = pd.read_csv('../data/station.csv',
                   dtype={'HUCEightDigitCode':'str'})

In [ ]:
#Display the dimensions of the data frame
sites.shape

In [ ]:
#Display the columns and their data types
sites.dtypes

### Exploring the data: Plot the sites
Plot a map to see where water quality data were present from this portal.<br> First we'll plot all the points, then we'll remove sites with small drainage areas and see how many remain...

In [ ]:
#Plot all sites--

#Find center coordinates from medians of lat and long columns
medLat = sites['LatitudeMeasure'].median()
medLng = sites['LongitudeMeasure'].median()

#Create the initial map
m = folium.Map(location=[medLat,medLng],
               zoom_start=9,
               tiles='stamenterrain')

#Loop through all features and add them to the map as markers
for row in sites.itertuples():
    #Get info for the record
    lat = row.LatitudeMeasure
    lng = row.LongitudeMeasure
    name = row.MonitoringLocationName
    #Create the marker object, adding them to the map object
    folium.CircleMarker(location=[lat,lng],
                        popup=name,
                        color='red',
                        fill=True,
                        fill_opacity=0.6,
                        radius=3,
                        stroke=False).add_to(m)
    
#Show map
m

In [ ]:
#Plot sites with an area > 25 sq mi
sites2 = sites[sites['DrainageAreaMeasure/MeasureValue'] > 25]
sites2.shape

*Only 10 records!* This seems small. Let's plot these in blue on top of the map created above to see where these 10 sites occur. 

In [ ]:
#Loop through all features and add them to the map as markers
for row in sites2.itertuples():
    #Get info for the record
    lat = row.LatitudeMeasure
    lng = row.LongitudeMeasure
    name = row.MonitoringLocationName
    #Create the marker object, adding them to the map object
    folium.CircleMarker(location=[lat,lng],
                        popup=name,
                        color='blue',
                        fill=True,
                        fill_opacity=0.8,
                        radius=3,
                        stroke=False).add_to(m)
    
#Show map
m

We take a closer look at the drainage measurement values and see there are numerous `NA` values. *Clearly, filtering by drainage area will not work*. The main point I want to make here is <mark>**don’t be afraid to play with the data.**</mark>

Let’s replot the site map but now change the popup name to see which site names are located within each branch of Jordan Lake.

In [ ]:
#Show the map
m = folium.Map(location=[medLat,medLng],
               zoom_start=9,
               tiles='stamenterrain')

#Loop through all features and add them to the map as markers
for row in sites.itertuples():
    #Get info for the record
    lat = row.LatitudeMeasure
    lng = row.LongitudeMeasure
    name = row.MonitoringLocationIdentifier #<-- Change the field to show as the popup
    #Create the marker object, adding them to the map object
    folium.CircleMarker(location=[lat,lng],
                        popup=name,
                        color='red',
                        fill=True,
                        fill_opacity=0.6,
                        radius=3,
                        stroke=False).add_to(m)
    
#Show map
m

Finally, let's cull some unused columns from our dataset

In [ ]:
keepColumns = [0,2,3,7,8,11,12]
sites = sites.iloc[:,keepColumns]
sites.columns

---
## Import and explore the water quality *measurement* data
The second dataset downloaded from the [Water Quality Data portal](https://www.waterqualitydata.us/portal/) were the water quality results, stored in the [`result.csv`](https://www.waterqualitydata.us/Result/search?countrycode=US&statecode=US%3A37&siteType=Lake%2C%20Reservoir%2C%20Impoundment&siteType=Stream&huc=03030002&sampleMedia=Water&characteristicType=Nutrient&minresults=500&startDateLo=01-01-1970&startDateHi=12-31-2017&mimeType=csv&zip=yes&sorted=no) file. Let's now import this \[large\] file and tidy it up for analysis.

In [ ]:
#Read in the results.csv
results = pd.read_csv('../data/result.csv',
                      low_memory=False      #This is required as it's a large file...
                     )

In [ ]:
#Check the dimensions of the dataframe: it's BIG
results.shape

### Clean the data by filtering records to only the ones we want
Once we load in the measurements data, we need to do some cleaning. The first step is to filter the data to what we want to use. For example,
* We want to restrict our analysis to streams, filtering out all other media.<br><br>

* We want to make sure we are using routine samples and not extreme event sampling (biased for specific occasions and not for estimating annual average load). So, we'd like to filter out events like storms, droughts, floods, and spring break ups from our analysis.<br><br>

* We determine what type of Nitrogen we want to use. From the literature we found that regulations for Nitrogen include: nitrate, nitrite, ammonia and organic forms. Doing some reading about the WQX standards, you learn that Nitrogen, mixed froms incorporates all of the above forms of nitrogen.<br><br>

*  We also want to make sure we are looking at *total nitrogen*, so we want to make sure the `Results Sample Fraction Text` only includes those with `Total`.

It's handy to know the to what each field in our data refers and what it's values are. An easy way to inspect the latter is with the Pandas `unique()` function which lists the unique values found in a give column...

In [ ]:
#List the unique values inthe HydrologicEvent column
results['HydrologicEvent'].unique()

In [ ]:
#List the unique values inthe CharacteristicName column
results['CharacteristicName'].unique()

Filtering in Pandas can be done by creating bitwise (i.e. True/False) "mask" and then combining them with logical operations..

In [ ]:
#Step 1 in filtering: creating a mask for each criteria
mediaMask = results['ActivityMediaSubdivisionName'] == "Surface Water"
hydroMask = ~results['HydrologicEvent'].isin(("Storm","Flood","Spring breakup","Drought")) 
charMask = results['CharacteristicName'] == "Nitrogen, mixed forms (NH3), (NH4), organic, (NO2) and (NO3)"
sampFracMask = results['ResultSampleFractionText'] == 'Total'

In [ ]:
#Step 2 in filtering: Applying the masks using logical combinations
nitrogen = results[mediaMask & hydroMask & charMask & sampFracMask] 
nitrogen.shape

In [ ]:
#Check that the filters worked
nitrogen['ActivityMediaSubdivisionName'].unique()

In [ ]:
nitrogen['HydrologicEvent'].unique()

In [ ]:
nitrogen['CharacteristicName'].unique()

In [ ]:
nitrogen['ResultSampleFractionText'].unique()

In [ ]:
#Subset columns
usecols=[0,6,21,30,31,32,33,34,58,59,60]
nitrogen = nitrogen.iloc[:,usecols]
nitrogen.columns

## Detection Limits and Unit Conversion
You may have noticed that many sample sites state “not detected”. This is important data that are not currently being represented. Create a new column and set the value equal to the results, unless it is below the detection limit - in which case set it equal to ½ of the detection limit.

You may also have noted that the total nitrogen was sometimes reported as mg/l or mg/l NO3. We want mg/l. To convert to mg/l, we know the atomic weight of nitrogen is 14.0067 and the molar mass of nitrate anion (NO3) is 62.0049 g/mole. Therefore, to convert between units:
* Nitrate-N (mg/L) = 0.2259 x Nitrate-NO3 (mg/L)
* Nitrate-NO3 (mg/L) = 4.4268 x Nitrate-N (mg/L)

In [ ]:
#Set data below the detection limit equal to 1/2 the detection limit

#Set default values to the ResultMeasureValue
nitrogen['TotalN'] = nitrogen['ResultMeasureValue']
nitrogen['TotalN'].mean()

In [ ]:
#Create a mask of values that are "Not Detected" and update those valuse
ndMask = nitrogen['ResultDetectionConditionText'] == 'Not Detected'
nitrogen.loc[ndMask,"TotalN"] = nitrogen['DetectionQuantitationLimitMeasure/MeasureValue'] / 2
nitrogen['TotalN'].mean()

In [ ]:
#Convert mg/l as NO3 to mg/l as N
no3Mask = nitrogen['DetectionQuantitationLimitMeasure/MeasureUnitCode'] == 'mg/l NO3'
nitrogen.loc[no3Mask,'TotalN'] = nitrogen['TotalN'] * 0.2259

### Export our saved records

In [ ]:
nitrogen.to_csv("../data/NResults.csv",index=False)

# Exploratory Data Analysis

## Merge sites and measurements together
Currently, site and measurement data are not connected together. However, we may want to show the nitrate values on a map. To do this, we merge the data together based on a unique identifier shared between the two data sets. In this case, it is the *MonitoringLocationIdentifier* column.

In [ ]:
#Merge the two datasets, joining the sites to the results (as sites may have >1 result) 
nitrodata = pd.merge(left=sites,
                     right=nitrogen,
                     how='right',
                     on='MonitoringLocationIdentifier')
nitrodata.shape

In [ ]:
#Convert the ActivityStartDate to a datetime object
nitrodata['ActivityStartDate'] = pd.to_datetime(nitrodata['ActivityStartDate'],format="%Y-%m-%d")

In [ ]:
#Create year values from the ActivityStartDate column
nitrodata['Year'] = pd.DatetimeIndex(nitrodata['ActivityStartDate']).year
nitrodata.groupby('Year')['Year'].count()

In [ ]:
#Create month categories from the ActivityStartDate column 
nitrodata['Month'] = pd.DatetimeIndex(nitrodata['ActivityStartDate']).month

In [ ]:
nitrodata['Month'].plot(kind='hist',
                        bins=12,
                        figsize=(8,6));

### Filter merged data

First we want to know how many sites are collecting data of interest. We find there are 22 sites. Of those sites, we want to know which were collecting data after the Jordan Lake Rules were passed and how much data are being collected at this site.

Based on this, we see several sites stopped collecting data prior to 2009, when the first iteration of Jordan Rules were passed. We also see that some of these sites collected only a few years of data. Remove those sites and plot the remaining sites on the map. 

In [ ]:
#Tally the number of unique sites
nitrodata['MonitoringLocationIdentifier'].nunique()

In [ ]:
#Group data by site
siteGroup = nitrodata.groupby('MonitoringLocationIdentifier')

#Compute min and max of year
siteInfo = siteGroup.agg({'Year':('min','max')})

#Rename Colummns
siteInfo.columns = ['StartYear','EndYear']

#Compute Range
siteInfo['Range'] = siteInfo['EndYear'] - siteInfo['StartYear']
siteInfo.head()

In [ ]:
#Limit data to those with more than 10 years data and still operating
siteInfo = siteInfo.query('Range >= 10 & EndYear >= 2017')
siteInfo

In [ ]:
#Filter our compined table to contain only these sites
siteMask = nitrodata['MonitoringLocationIdentifier'].isin(siteInfo.index)
df = nitrodata[siteMask]
df.shape

In [ ]:
#Group and map these sites
df2 = df.groupby('MonitoringLocationIdentifier').first()

#Find center coordinates from medians of lat and long columns
medLat = df2['LatitudeMeasure'].median()
medLng = df2['LongitudeMeasure'].median()

#Create the initial map
m = folium.Map(location=[medLat,medLng],
               zoom_start=9,
               tiles='stamenterrain')

#Loop through all features and add them to the map as markers
for row in df2.itertuples():
    #Get info for the record
    lat = row.LatitudeMeasure
    lng = row.LongitudeMeasure
    name = row.MonitoringLocationName
    #Create the marker object, adding them to the map object
    folium.CircleMarker(location=[lat,lng],
                        popup=name,
                        color='red',
                        fill=True,
                        fill_opacity=0.8,
                        radius=5,
                        stroke=False).add_to(m)
    
#Show map
m

### Plot total nitrogen over time

### Plot the 2017 Nitrogen year on leaflet

## Compare the Nitrogen Load with Thresholds
The water quality data reports Nitrogen as mg/l. In order to convert to an annual load (lbs/yr), we need to know the volume of water flowing through each site. Go to the NWIS Mapper to find which USGS gauges are closest to the Haw River Arm (1 site) and the New Hope Arm (3 sites).

### Load NWIS data for Haw River

### Calculate Annual Load for Haw River

To calculate the annual load we need to convert cfs to MGD. Then we use pipes and dplyr to calculate the total annual flow at Haw River. Next, we calculate the average Nitrogen load for samples taken during each year. This is a rough proxy. A finer analysis can be undertaken by summarizing monthly flow and water quality, aggregating to the year as the last step. The annual load is then the: Total Flow * average Nitrogen * 8.34 lbs per gallon

We can then plot the annual load with the threshold of 2.567 Million pounds per year.

### Calculate Annual Load for New Hope Creek

Here there are 3 upstream gauges for New Hope Creek. We will download that information into a single file. We repeat the above analyis with the New Hope stream gauges.